In [9]:
import re 
from time import time
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#vieja formulacion sin las binarias, todas las variables continuas

#-------------------nombre del archivo a leer--------------------

fic = open( "datos.txt" , "r" )

#----------------------------------------------------------------
#----------------------------Leer datos--------------------------
#----------------------------------------------------------------
#nota: esto puede leer los datos de cualquier archivo que cumpla 
#el formato sin importar el numero de estaciones de la red

Slabel = re.compile( r'[A-Z][0-9]+' )
Costos = re.compile( r'[0-9]+' )
u = 0
m = 0
Q = 0 
rutas = 247
lines = [ ]
for line in fic:
    lines.append( line )
    
#leer Slabel

i = 0
while( lines[i] != "Slabel:[\n" ):
    i = i + 1
i = i + 1 
nodos2 = { }
j = i
while( lines[ i ] != "]\n" ) :
    result = Slabel.search( lines[ i ] )
    i = i + 1 
    nodos2[ i - j ]= result.group()
i = i + 1  

#leer Dlabel

while( lines[ i ] != "Dlabel:[\n" ):
    i = i + 1
i = i + 1  
while( lines[ i ] != "]\n" ) :
    i = i + 1 
i = i + 1 

#leer costos

c2 = { }
while( lines[ i ] != "cost:[\n" ):
    i = i + 1
i = i + 1  
while( lines[ i ] != "]\n" ) :
    result = Costos.findall( lines[ i ] ) 
    i = i + 1
    c2[ ( int( result[ 0 ] ) , int( result[ 1 ] ) ) ] = int( result[ 2 ] )
i = i + 1 

#leer u

while( lines[ i ][ 0 ] != "u" ):
    i = i + 1
result = Costos.findall( lines[ i ] )   
u = int( result[ 0 ] )
i = i + 1

#leer m

while( lines[ i ][ 0 ] != "m" ):
    i = i + 1
result = Costos.findall( lines[ i ] )   
m = int( result[ 0 ] ) 
i = i + 1  

#leer Q

while( lines[ i ][ 0 ] != "Q"):
    i = i + 1
result = Costos.findall(lines[i])   
Q = int( result[ 0 ] )
i = i + 1   

#leer stat

stat = { }
while( lines[ i ] != "stat:[\n" ):
    i = i + 1
i = i + 1  
while( lines[ i ] != "]\n" ) :
    result = Costos.findall( lines[ i ] ) 
    i = i + 1 
    stat[ int( result[ 0 ] ) ] = int( result[ 1 ] ) 
i = i + 1 
B2 = { ( o , d , i ):0 for o in nodos2 for d in nodos2 for i in stat }

#leer b

while( lines[ i ] != "b:[\n" ):
    i = i + 1
i = i + 1  
while( lines[i] != "]\n" ) :
    result = Costos.findall( lines[ i ] ) 
    B2[ ( int( result[ 0 ] ) , int( result[ 1 ] ) , int( result[ 0 ] ) ) ] = int( result[ 2 ] )
    B2[ ( int( result[ 0 ] ) , int( result[ 1 ] ) , int( result[ 1 ] ) ) ] = ( -1 ) * int( result[ 2 ] )  
    i = i + 1 
i = i + 1 
Ar2 = { }
rutaAsociada = { }

#leer Ar

while( lines[ i ] != "Ar:[\n" ):
    i = i + 1
i = i + 1  
j = i
while( lines[i] != "]\n" ) :
    result = Costos.findall( lines[ i ] ) 
    i = i + 1 
    Ar2[ ( int( result[ 0 ] ) , int( result[ 1 ] ) ) ] = int( result[ 3 ] )
    rutaAsociada[ i - j ] = int( result[ 2 ] )
i = i + 1 
As2 = { }

#leer As

while( lines[ i ] != "As:[\n" ):
    i = i + 1
i = i + 1  
while( lines[ i ] != "]\n" ) :
  result = Costos.findall( lines[ i ] ) 
  i = i + 1 
  As2[ ( int( result[ 0 ] ) , int( result[ 1 ] ) ) ] = int( result[ 3 ] ) 
i = i + 1 

#matriz de costos nueva


C2 = { }
for i in Ar2:
    C2[ i ] = c2[ ( stat[ i [ 0 ] ] , stat[ i [ 1 ] ] ) ]  
for i in As2:   
    C2[ i ] = 0


In [11]:

#--------------------------------------------------------
#----------------------formulacion-----------------------
#--------------------------------------------------------


from docplex.mp.model import Model
mdl = Model( "modelo" )

#-------------------variables decision -------------------

print( "--------------------INICIANDO----------------")


arcos = [ ( i [ 0 ] , i [ 1 ] , o , d ) for i in C2 for o in nodos2 for d in nodos2]
x = mdl.continuous_var_dict( arcos , name = "x" )
arcosF = [ ( i ) for i in range( 1 , rutas + 1 ) ]
f=mdl.continuous_var_dict(arcosF,name="f")

#----------------------parametros-------------------------
#cree esta parte para que fuera mas facil cambiar estos
#parametros del problema sin tener que modificar el archivo

u= 90
m= 35
Q= 50
alpha=60
#--------------------------FO---------------------------
print( "----------------agregando FO----------------")
mdl.minimize( mdl.sum( C2[ i ] * x [ ( i [ 0 ] , i [ 1 ] , o , d ) ] for i in C2 for o in nodos2 for d in nodos2 ) + alpha * mdl.sum( f[ k ] for k in range( 1 , rutas + 1 ) ) )

#---------------------------------------------------------
#----------------------restricciones----------------------
#---------------------------------------------------------

#-------------------------flujo---------------------------
print( "---------agregando restricciones de flujo---------")
aux = { }
aux2 = { }
for i in stat:
    aux.clear()  
    aux2.clear()
    for j in C2:
        if( i == j[ 0 ]):
            aux[ j ] = C2[ j ]
    for j in C2:
        if( i == j[ 1 ] ):
            aux2[ j ] = C2[ j ]                       
    for o in  nodos2:
        for d in nodos2:       
            mdl.add_constraint( ( mdl.sum( x [ ( k [ 0 ] , k [ 1 ] , o , d ) ] for k in aux ) - mdl.sum( x [ ( k [ 0 ] , k [ 1 ] , o , d ) ] for k in aux2 ) ) == B2[ ( o , d ,i ) ] )

#------------------------capacidad--------------------------
print( "------agregando restricciones de capacidad------")
aux3 = { }
for k in range( 1 , rutas + 1 ):
    j = 1
    aux3.clear( )
    for i in Ar2:
        if( k == rutaAsociada[ j ] ):
            aux3[ i ] = 1            
        j = j + 1 
    for l in aux3:    
        mdl.add_constraint( mdl.sum( x [ ( l[ 0 ] , l[ 1 ] , o , d ) ] for o in nodos2 for d in nodos2 ) <= u * f[ ( k ) ] )  
        
#------------------maximo numero buses--------------------------


mdl.add_constraint( mdl.sum( f[ k ] for k in range( 1 , rutas + 1 ) ) <= Q )  

#---------------------------------------------------------
#------------------------solucionar-----------------------
#---------------------------------------------------------
#el tiempo de solucionar el probelma, mas no el de crear todas las variables,restricciones,etc.

start_time = time()
print( "--------------------SOLUCIONANDO----------------")  
solucion = mdl.solve( log_output = True )  

elapsed_time = time( ) - start_time
print( "tiempo: %0.10f seconds." % elapsed_time)    

#------------------------guardar solucion-----------------------

archivo = open("solucion_formulacion_vieja", "w") 
archivo.write(str(solucion)) 
archivo.close()

--------------------INICIANDO----------------
----------------agregando FO----------------
---------agregando restricciones de flujo---------
------agregando restricciones de capacidad------
--------------------SOLUCIONANDO----------------
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 24 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 22 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 16 rows and 3372 columns.
Aggregator did 448 substitutions.
Reduced LP has 83 rows, 555 columns, and 1357 nonzeros.
Presolve time = 0.00 sec. (2.70 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =           500.000000

Dual simplex solved model.

tiempo: 0.0330080986 seconds.


In [4]:
for i in stat:
    print(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
